In [ ]:
def time_domain_spectrum_model(parameters=[0.30,0.07,2800.,3900.],
                               order = None,
                               visit = 'define your visit in the time domain spectrum model!',
                               spotspec = None,
                               ambspec = None,
                               plot=False,**kwargs):
    if visit == 'fall':
        phase = 1.88
        T0 = (59455.48 + 2400000.5)
    if visit == 'spring':
        phase = 2.02
        T0 = (59455.48 + 2400000.5) + (27.*8.46321)
    T_rot = 4.863 # AU mic rotation period, days
    f_spot,df_spot,T_spot, T_amb = parameters
    spot_spec = spotspec
    amb_spec = ambspec
    contrast = 1.-(spot_spec/amb_spec)
    ds_over_s = -df_spot * ( contrast / ( 1.-f_spot * contrast ) )
    
    # Set up and calculate the sinusoidal S(lambda,t) model
    spec_model = np.ones_like(nres_rainbow.flux)
    for i in range(len(data_times)):
        S_t_model = 0.0
        ft_spot = f_spot - df_spot*np.sin(2*np.pi*data_times[i].value/T_rot + phase)
        combined = ft_spot*spot_spec + (1.-ft_spot)*amb_spec
        S_t_model = combined/np.median(combined)
        spec_model[:,i] = S_t_model
        
    chisq = np.nansum((nres_rainbow.flux - spec_model)**2/(nres_rainbow.uncertainty)**2)
    reduced_chisq = chisq/((len(data_wave)-4)*len(data_times))
    kludge = np.sqrt(reduced_chisq)

    if plot:
        model_data_times = np.linspace(data_times[0].value-1,data_times[-1].value+1,300)
        cmap = cm.get_cmap('cividis', len(data_times))
        colormap = cmap(np.linspace(0, 1, len(data_times)))

        fig, axs = plt.subplots(2,3,figsize=(18,10))
        title_label=f'Order {order} | fspot+/-'+r'$\Delta$'+f'fspot = {f_spot:.2f}+/-{df_spot:.2f} | Tspot: {int(T_spot/10)*10} | Tamb: {int(T_amb/10)*10} | err kludge: {kludge:0.1f} | reduced chisq: {reduced_chisq:.2f}'
        fig.suptitle(title_label,fontsize=18)

        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[0,1]
        ax4 = axs[1,1]
        ax5 = axs[0,2]
        ax6 = axs[1,2]

        for i in range(len(data_times)):
            ax3.errorbar(data_wave,nres_rainbow.flux[:,i],yerr=nres_rainbow.uncertainty[:,i],alpha=0.6,
                         color=colormap[i],fmt='')
            ax4.errorbar(data_wave,(spec_model[:,i]-nres_rainbow.flux[:,i])/nres_rainbow.uncertainty[:,i],
                     yerr = 1,alpha=0.6,color=colormap[i],fmt='|')
        
        ax1.set_title(r'Model dS/dT',fontsize=18)
        ax1.plot(data_wave,(spot_spec/np.median(spot_spec)-amb_spec/np.median(amb_spec))/(T_amb-T_spot),color='k')
        ax1.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax1.set_ylim(0.000001,0.1)
        ax1.set_ylabel('dS/dT',fontsize=18)
        ax1.set_yscale('log')
        
        ax2.set_title(r'Model $\Delta$S/S',fontsize=18)
        ax2.plot(data_wave,np.abs(ds_over_s))
        ax2.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax2.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax2.set_ylim(0,0.1)
        ax2.set_ylabel(r'$\Delta$S/S',fontsize=18)        

        ax3.set_title('NRES data in time',fontsize=18)
        ax3.plot(data_wave, np.median(spec_model,axis=1),color='k',zorder=10,label='median model spectrum')
        ax3.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax3.set_ylim(0.5,1.5)
        ax3.set_ylabel('Relative flux',fontsize=18)
        ax3.legend()
           
        ax4.set_title(r'Residuals',fontsize=18)
        ax4.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax4.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax4.set_ylabel(r'$\sigma$',fontsize=18)

        ax5.set_title(r'NRES S($\lambda$,t)/S($\lambda$)',fontsize=18)
        speccompplot = ax5.imshow(np.transpose(nres_spec_comparison),aspect='auto',cmap='Reds',
                                  extent=[data_wave[0].value,data_wave[-1].value,data_times[-1].value,data_times[0].value])
        ax5.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax5.set_ylabel('BJD',fontsize=18)
        plt.colorbar(speccompplot, ax = ax5)
        
        ax6.set_title('PHOENIX Model Spectra',fontsize=18)
        specplot= ax6.imshow(np.transpose(spec_model),aspect='auto',cmap='Reds',
                             extent=[data_wave[0].value,data_wave[-1].value,data_times[-1].value,data_times[0].value])
        ax6.set_ylabel('BJD',fontsize=18)
        ax6.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        plt.colorbar(specplot, ax = ax6) 

        if do_TimeSpec:
            plt.savefig(f'figs/Timeseries_models/{visit}_{order}_{file_label}_timeseriesspectra.png',dpi=300)
        plt.clf()

    return spec_model,chisq